# Homework 6 - Text Generation

Barack Obama is a politician who served as the 44th President of the United States from 2009 to 2017. During that time, his speech writers have produced over 4.3MB or 730,895 words of text, not counting interviews and debates. All of Obama’s speeches are available at (http://www.americanrhetoric.com/barackobamaspeeches.html)

Fortunately, samim has already scraped Obama's speeches. They are available at https://github.com/samim23/obama-rnn/.

In this homework, you will try to replicate Obama's speech using a one-to-many model on **WORD LEVEL**. you are asked to do the following tasks:

    Preprocessing data for keras
    Writing a one-to-many model on **WORD LEVEL**
    Generating somewhat Obama-liked speech
   
As some of you may realize, the tutorial code may not be sufficient for this dataset due to the sheer volume of the input data. You should refer to keras tutorial in text generation for training strategy for large text. (https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py)    

## Preprocessing

## TODO1

Download and Preprocess Obama speech as you see fit. (**Tip: NLTK word tokenizer**)

In [1]:
# Write your code here
# You may create additional cells after this cell for your preprocess
import csv
import numpy as np
import pandas as pd
import random
import math
import datetime
import sys
import nltk
from nltk.tokenize import word_tokenize
import collections

from keras.callbacks import LambdaCallback, ModelCheckpoint
from keras.backend import argmax,one_hot,tf
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Reshape, Input, Lambda
from keras.layers import SimpleRNN, Masking, Embedding
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

/home/ekapolc/.env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
nltk.download('punkt')

In [2]:
data = []
with open('input.txt', 'r') as f:
    for l in f:
        if(l.strip() != ''):            
            data.append(l.strip().lower())            
#data_size, vocab_size = len(data), len(chars)

In [3]:
maxlen = 0
minlen = 1000
tokened_data = []
for l in data:
    token = word_tokenize(l)
    maxlen = max(maxlen, len(token))
    minlen = min(minlen, len(token))
    tokened_data.append(token)

In [4]:
print("max length =",maxlen)
print("min length =",minlen)
print(tokened_data[0])

max length = 391
min length = 2
['to', 'chip', ',', 'kathy', ',', 'and', 'nancy', ',', 'who', 'graciously', 'shared', 'your', 'father', 'with', 'a', 'nation', 'that', 'loved', 'him', ';', 'to', 'walter', "'s", 'friends', ',', 'colleagues', ',', 'protégés', ',', 'and', 'all', 'who', 'considered', 'him', 'a', 'hero', ';', 'to', 'the', 'men', 'of', 'the', 'intrepid', ';', 'to', 'all', 'of', 'you', 'who', 'are', 'gathered', 'here', 'today', ';', 'i', 'am', 'honored', 'to', 'be', 'here', 'to', 'pay', 'tribute', 'to', 'the', 'life', 'and', 'times', 'of', 'the', 'man', 'who', 'chronicled', 'our', 'time', '.']


In [5]:
def create_index(input_text):
    count_word = 0
    words = []
    
    for sent in input_text:
        for w in sent:
            words.append(w.strip('\n'))
            count_word +=1
    
    word_count = list()
    #use set and len to get the number of unique words
    word_count.extend(collections.Counter(words).most_common(len(set(words))))
    
    #include a token for unknown word
    threshold = 20
    num_UNK = 0
    index = len(word_count) - 1
    rare_word = set()
    
    
    while(word_count[index][1] <= threshold):
        num_UNK += word_count[index][1]
        rare_word.add(word_count[index][0])
        index -= 1
    
    word_count = word_count[:index+1]
    word_count.append(("<UNK>",num_UNK))
    word_count = sorted(word_count, key=lambda x: -x[1])
    
    print(count_word)
    print(num_UNK , num_UNK/count_word, len(rare_word)/len(word_count))       

    #print out 10 most frequent words
    
    print(word_count[:10])
    dictionary = dict()
    dictionary["for_keras_zero_padding"] = 0
    dictionary['<S>'] = 1
    for word in word_count:
        dictionary[word[0]] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    
    print("vocab size =",len(dictionary))
    
    dataset = list()
    for sent in input_text:
        dataset.append([])
        for word in sent[:maxlen]:
            if(word not in rare_word):
                dataset[-1].append(dictionary[word])
            else:
                dataset[-1].append(dictionary["<UNK>"])
    return dataset, dictionary, reverse_dictionary, len(dictionary)

dataset ,word_to_ix,ix_to_word,vocab_size = create_index(tokened_data)

848455
61151 0.07207335686630405 5.822592067988668
[('<UNK>', 61151), (',', 37387), ('.', 37358), ('the', 34744), ('and', 27758), ('to', 25296), ('of', 19970), ('that', 18381), ('we', 14779), ('a', 14508)]
vocab size = 2826


In [6]:
data_size = len(dataset)
print("data_size" ,len(dataset))
print("vocab_size", vocab_size)

data_size 10476
vocab_size 2826


In [ ]:
%%time
# read glove
glove = {}
with open('/data/glove.840B.300d.txt','r') as f:
    for line in f:
        splitLine = line.split(' ')
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        glove[word] = embedding

In [ ]:
match = 0
for w in word_to_ix.keys():
    if(w.strip('\n') in glove.keys()):
        match += 1

print('match', match, match/len(word_to_ix))

In [ ]:
### Prepare embed layer
def prepare_embed(pretrain):
    pre_emb = []
    pre_emb.append(np.zeros(300))

    for k in word_to_ix.keys():
        if(not k in pretrain.keys()):    
            pre_emb.append(np.zeros(300))
        else:
            if(len(pretrain[k]) ==  300):
                pre_emb.append(pretrain[k])
            else:
                pre_emb.append(np.zeros(300))
    return np.array(pre_emb)

pre_emb = prepare_embed(glove)
print(pre_emb.shape)

In [ ]:
import pickle
with open('/data/embed-text-gen-50', 'wb') as f:
    pickle.dump(pre_emb, f)

In [ ]:
import pickle
# with open('/data/embed-text-gen', 'rb') as f:
#     pre_emb = pickle.load(f)
    
with open('/data/embed-text-gen-50', 'rb') as f:
    pre_emb = pickle.load(f)

In [8]:
#Preparing output data for the model

maxlen = 40
Y = dataset
# for line in dataset:
#     Y.append(line[:maxlen])
# pre_Y = Y    
Y = pad_sequences(dataset, truncating='post', padding = 'post', maxlen=maxlen).tolist()

#Preparing input data for the model
#The first element of the sequence is an initial seed
#The rest is just like Y but shifted by one-time-step 
X = []
for item in Y:
    X.append([1] + item[:-1]) #Add initial seeed and shift X by one time step
# pre_X = X        

#Preparing data for Keras
X= to_categorical(X,vocab_size) #one-hot
Y= to_categorical(Y,vocab_size)
X=X.reshape(data_size,maxlen ,vocab_size)
Y=Y.reshape(data_size,maxlen ,vocab_size)
Y= np.swapaxes(Y,0,1)
print(np.array(X).shape, np.array(Y).shape)

(10476, 40, 2826) (40, 10476, 2826)


## TODO2

Describe your data and preprocessing methods here.

    What is your input (X,Y) ? What is its shape?
    How do you preprocess your data?



## Model

## TODO3

Create Keras model for **word-level** speech generation. Your must use 'accuracy' as one of your metric.

Your best model accuracy should be at least 0.15.

In [9]:
## Write your text generation model here
#Shared layers (Global)
n_a = 16 #number of hidden dimensions
reshapor = Reshape((1, vocab_size)) #Reshape the size of a tensor                         
RNN_cell = SimpleRNN(n_a, return_state = True) #An RNN Cell       
output_layer = Dense(vocab_size, activation='softmax')  #softmax output layer

In [10]:
def train_model(Tx, n_a, n_values):
    """
    Implement the model for the training phase
    
    Arguments:
    Tx -- length of the sequence 
    n_a -- the number of hidden dimensions used in our model
    n_values -- number of unique labels in the data 
    
    Returns:
    model -- a keras model instance
    """
    
    # Define the input of your model
    input1 = Input(shape=(Tx, n_values))
#     print(input1.shape)
#     X = Embedding(vocab_size+1, 300, weights=[pre_emb], trainable=True, mask_zero=True)(input1)
    X = input1
#     print(X.shape)
    
    # Define a0, initial hidden state for the RNN
    a0 = Input(shape=(n_a,), name='a0')
#     a = Masking(mask_value=0)(a0)
    a = a0
    # Create empty list to append an output from the model in each loop
    outputs = list()
#     print("{}\n".format(a.shape))
    
    # Loop  through the sequence of length Tx
    for t in range(Tx):
        # Select the "t"th time step vector from X. 
        x =  Lambda(lambda x: X[:,t,:])(X)
        # Reshape x to be (1, n_values)
#         print(x.shape)
        x = reshapor(x) 
        # Update the hidden state of the RNN 
#         print(x.shape, a.shape)
        a, _ = RNN_cell(x, initial_state=[a]) 
        # Pass the hidden vector to a softmax function
        out = output_layer(a)
        # Append an output list with the current output
        outputs.append(out)
        
    # Create the model instance
    model =  Model(inputs=[input1,a0], outputs=outputs)    
    return model

In [11]:
model = train_model(Tx = maxlen , n_a = n_a, n_values = vocab_size)
opt = Adam(lr=0.001) #optimizer
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

In [12]:
def inference_model(RNN_cell, output_layer, n_values = vocab_size ,n_a = n_a, Ty = 500):
    """
    Implement the model for inferencing/testing phase using the parameters learned from the previous steps
    
    Arguments:
    RNN_cell -- the trained "RNN_cell" from train_model(), Keras layer object
    output_layer -- the trained "output_layer" from train_model(), Keras layer object
    n_values -- mumber of unique characters
    n_a -- number of dimensions in RNN_Cell
    Ty -- number of time steps to generate
    
    Returns:
    inference_model -- Keras model instance
    """
    
    # Define the input of your model
    x0 = Input(shape=(1, n_values))
#     x0 = Embedding(vocab_size+1, 300, weights=[pre_emb], trainable=True, mask_zero=True)(input1)

    # Define a0, initial hidden state for the decoder RNN
    a0 = Input(shape=(n_a,), name='a0')
    
    a = a0
    x = x0
    
    print("{} {}".format(x.shape, a.shape))

    # Create an empty list of "outputs" to stored the predicted outputs
    outputs = list()
    
    #Loop over Ty and generate a value at each time step
    for t in range(Ty):
        
        # Perform one step of RNN_cell
#         print("{} {}".format(x.shape, a.shape))
        a, _ = RNN_cell(x, initial_state=[a])
        
        # Apply Dense softmax layer to the hidden state output of the RNN_cell
        out = output_layer(a)
#         print("{}".format(out.shape))

        # Append an output list with the current output
        outputs.append(out)
        
        # Sample the new value to pass to the next time step
        #tf.log because tf.multinomail wants unnormalized log-prob inputs
        x  = Lambda(lambda x: one_hot(tf.multinomial(tf.log(x), 1), num_classes=n_values))(out)
        x = Reshape((1,n_values))(x)
  
    #Create the model instance
    inference_model = Model(inputs=[x0,a0], outputs=outputs)
    
    return inference_model

In [13]:
inference_model = inference_model(RNN_cell, output_layer, n_values = vocab_size, n_a = n_a, Ty = 500)

(?, 1, 2826) (?, 16)


In [14]:
x_initializer = np.zeros((1, 1, vocab_size))
a_initializer = np.zeros((1, n_a))
# inference_model.summary()

In [15]:
x_initializer[0][0][word_to_ix["<S>"]]=1 #initial seed
print(np.argmax(x_initializer[0][0]))

1


## TODO4

Write code for text generation part. Speech should be generated every 5 epochs while training, and with at least 500 words length.

In [18]:
from keras.callbacks import LambdaCallback

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def generate_text(inference_model, x_initializer = x_initializer, a_initializer = a_initializer):
    """
    generate a date using the inference model.
    
    Arguments:
    inference_model -- Keras model instance for inference/test time
    x_initializer -- numpy array of shape (1, 1, 12), one-hot vector initializing the values generation
    a_initializer -- numpy array of shape (1, n_a), initializing the hidden state of the RNN_cell
    
    Returns:
    text -- a generated text string
    indices -- numpy-array of shape (Ty, 1), matrix of indices representing the values generated
    """
    
    #predict
    pred = inference_model.predict([x_initializer,a_initializer])
#     print(pred)
    #turn predictions into integers
    indices = np.argmax(pred,axis=-1)
#     indices = sample(pred)
    #intergers to text
    text = " ".join([ix_to_word[r[0]] for r in indices])
    
    return text


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.    
    if(epoch%10==0):
        print()
        global start_time
        t = datetime.datetime.now()
        print('### elaped', (t-start_time).seconds, 'seconds ###')
        start_time = t
        print('----- Generating text after Epoch: %d' % epoch)
        for i in range(3):
            text = generate_text(inference_model, x_initializer, a_initializer)
            sys.stdout.write(text)
            sys.stdout.flush()
            print()
    
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)        
checkpoint_callback = ModelCheckpoint(
    '/data/gentext.h5',
    monitor = "val_loss",
    mode = 'min',
    verbose = 1,
    save_best_only = True,
    save_weights_only = True,
)

In [ ]:
def get_indexed(x,a0,y,maxlen,num_classes):
    x = to_categorical(x,num_classes) #one-hot
    y = to_categorical(y,num_classes)
    x = x.reshape(1,len(x),num_classes)
    y = y.reshape(1,len(y),num_classes)
    print(x.shape, y.shape)
    return [x,a0],y

In [ ]:
a = np.array([[1,2],[3,4]])
(a.reshape(1,2,2)).shape

In [ ]:
# batch_size = 32
def get_training_data_generator(X,a0,Y, maxlen, num_classes):
    for i in range(len(X)):
        yield get_indexed(X[i], a0, Y[i], maxlen, num_classes)

In [ ]:
generator = get_training_data_generator(X,a0,Y,maxlen, vocab_size)

## TODO5

Generate speech while training. Select three best speeches that represent your models progression. For example,

**Epoch 1**

war the we . that 
 . that and the to to the to
...

**Epoch 50**

war we last if change a partnership he employees one of of , jobs now loss 
...

**Epoch 100**

war , and the pursuit of people who were lost


In [19]:
%%time
global start_time 
start_time = datetime.datetime.now()
m = data_size
a0 = np.zeros((m, n_a))
model.fit([X, a0], list(Y),batch_size=512,verbose=0 ,epochs=100,callbacks=[print_callback, checkpoint_callback])
# model.fit_generator(generator, steps_per_epoch=1,verbose=1, epochs=100, callbacks=[print_callback, checkpoint_callback])


### elaped 22 seconds ###
----- Generating text after Epoch: 0
<UNK> of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of 

/home/ekapolc/.env/lib/python3.5/site-packages/keras/callbacks.py:405: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


## Extra Score (1.8 points)

What we have used so far is a greedy algorithm, choosing word with highest problability, or choosing word using a random problability over word distribution. However, these approaches are far from optimal. As you have learned in POS Tagging lecture, it is possible to label the best POS tags over a sequence using beam search. 

You may read about how beam search can be used in text generation here (https://cs.stanford.edu/~zxie/textgen.pdf) 

Write additional code for text generation using beam search. 

Compare your results between text generation using greedy approach and beam search.

In [ ]:
# Write beam search function here

Compare your results here.